In [57]:
import pandas as pd

In [58]:
#Here changing fixing release dates column and removing the dates column
#This is fine. So no need to check. We need to merge this df with the imdb df. 
rotten_df=pd.read_csv("new_data.csv")
rotten_df.drop(['Rank'],axis=1,inplace=True)
for i in rotten_df.index:
    rotten_df['Release Date'][i]=rotten_df['Release Date'][i].replace('2021',str(rotten_df['Year'][i]))
rotten_df.drop(['Year'],axis=1,inplace=True)
rotten_df.drop_duplicates(subset=['Title'],keep='first',inplace=True)

C:\Users\ayush\AppData\Local\Temp/ipykernel_30956/2111572487.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rotten_df['Release Date'][i]=rotten_df['Release Date'][i].replace('2021',str(rotten_df['Year'][i]))


In [59]:
#rating_imdb is perfectly ok. We just need to match the tconst with the other dataset title_imdb
rating_imdb=pd.read_csv("rating_imdb.tsv", sep='\t')
title_imdb=pd.read_csv("title_imdb.tsv",sep='\t')

C:\Users\ayush\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [60]:
#We are removing the unnecessary fields. I took only the movies for regions for US.
#I took the first tconst value if it is repeated. titleID is later on changed to tconst. 

title_imdb.drop_duplicates(subset=['titleId'],keep='first',inplace=True)
title_imdb.drop(['ordering','types','attributes','isOriginalTitle','language'],axis=1,inplace=True)
title_imdb = title_imdb.rename({'titleId': 'tconst','title':'Title'}, axis=1)
title_imdb = title_imdb[title_imdb['region']=='US']

In [61]:
final_imdb=rating_imdb.merge(title_imdb, how='inner', on='tconst')

In [62]:
final_imdb.sort_values(by=['averageRating'])
final_imdb.drop_duplicates(subset=['Title'],keep='last',inplace=True)

In [63]:
finals_imdb=rotten_df.merge(final_imdb, how='inner', on='Title')

In [64]:
finals_imdb.to_csv('final_data.csv',index=True)

In [70]:
remove_missing = pd.read_csv("final_data.csv")
remove_missing["Score_Rotten"] = pd.to_numeric(remove_missing["Score_Rotten"],errors='coerce')
remove_missing["Score_Audience"] = pd.to_numeric(remove_missing["Score_Audience"],errors='coerce')

In [71]:
df = remove_missing.copy()

In [72]:
for i in df.index:
    counter_=0
    sum_=0
    if (pd.isna(df['Score_Rotten'][i])==True):
        date = df['Release Date'][i][-4:]
        for j in df.index:
            if (df['Release Date'][j][-4:]==date):
                counter_=counter_+1
                if (pd.isna(df['Score_Rotten'][j])==True):
                    pass
                else:
                    sum_=sum_+df['Score_Rotten'][j]
        df['Score_Rotten'][i]=int(sum_/counter_)

C:\Users\ayush\AppData\Local\Temp/ipykernel_30956/3936405554.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Score_Rotten'][i]=int(sum_/counter_)


In [73]:
for i in df.index:
    counter_=0
    sum_=0
    if (pd.isna(df['Score_Audience'][i])==True):
        date = df['Release Date'][i][-4:]
        for j in df.index:
            if (df['Release Date'][j][-4:]==date):
                counter_=counter_+1
                if (pd.isna(df['Score_Audience'][j])==True):
                    pass
                else:
                    sum_=sum_+df['Score_Audience'][j]
        df['Score_Audience'][i]=int(sum_/counter_)

C:\Users\ayush\AppData\Local\Temp/ipykernel_30956/600381596.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Score_Audience'][i]=int(sum_/counter_)


In [74]:
df.to_csv('last_final_data.csv',index=True)